In [1]:
from __future__ import absolute_import, division, print_function

import ast
import numpy as np

from rmexp import dbutils, config
from rmexp.schema import models
from logzero import logger



In [2]:
# get the average processing time and
# data size for different partition
exps = ['lego-cloudlet002-2core', 'lego-nexus6']
names = ['lego-tr{}'.format(idx) for idx in range(1, 6)]
sample_frame_ds = 691341
for exp in exps:
    for name in names:
        logger.info('==============={} {}==============='.format(exp, name))
        sess = dbutils.get_session()
        data = [[ast.literal_eval(item.speed), ast.literal_eval(item.data_length)]
                for item in sess.query(models.AppProfile.speed,
                                       models.AppProfile.data_length).filter(
                    models.AppProfile.name == name).filter(
                    models.AppProfile.exp == exp).all()]
        sess.close()
        pt, ds = list(zip(*data))
        pt = np.array(pt)
        sample_pt = np.zeros((pt.shape[0], 1))
        pt = np.concatenate([sample_pt, pt], axis=1)
        if 'cloudlet' in exp:
            # cloudlet
            pcts = np.cumsum(pt[:, ::-1], axis=1)[:, ::-1]
        else:
            # client
            pcts = np.cumsum(pt, axis=1)
        logger.info('mean: {}, std: {}'.format(
            np.average(pcts, axis=0), np.std(pcts, axis=0)))

        ds = np.array(ds)
        sample_ds = np.full((ds.shape[0], 1),
                            sample_frame_ds, dtype=np.int64)
        ds = np.concatenate([sample_ds, ds], axis=1)



[I 190507 17:07:14 <ipython-input-2-0424b31975a0>:9] ===============lego-cloudlet002-2core lego-tr1===============
[I 190507 17:07:14 <ipython-input-2-0424b31975a0>:28] mean: [ 122.50729087  122.50729087   85.91327705   11.33729854    4.78587874], std: [ 174.3535493   174.3535493   162.49994675   21.65407335    9.15104178]
[I 190507 17:07:14 <ipython-input-2-0424b31975a0>:9] ===============lego-cloudlet002-2core lego-tr2===============
[I 190507 17:07:14 <ipython-input-2-0424b31975a0>:28] mean: [ 195.81455399  195.81455399  158.4889336    20.89570758    8.73038229], std: [ 211.97579558  211.97579558  195.28543241   25.89820762   10.84265287]
[I 190507 17:07:14 <ipython-input-2-0424b31975a0>:9] ===============lego-cloudlet002-2core lego-tr3===============
[I 190507 17:07:14 <ipython-input-2-0424b31975a0>:28] mean: [ 189.89087093  189.89087093  150.17593564   18.79713186    8.13466247], std: [ 200.58071686  200.58071686  186.30706365   23.64992094   10.26356336]
[I 190507 17:07:14 <ipyth